In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets
from ipywidgets import interact, IntSlider
import torch
import os
from torch.utils.data import Dataset, DataLoader
from smplx import SMPLX

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
train_data = np.load('train_data_smplx.pkl', allow_pickle=True)
test_data = np.load('test_data_smplx.pkl', allow_pickle=True)

In [64]:
def convert_smplx_angle_rot(smplx_data):
    '''smplx_data: (T, 187)'''
    poses = smplx_data[:, 6: 171]
    poses = poses.reshape(poses.shape[0], 55, 3)
    magnitudes = np.linalg.norm(poses, axis=2, keepdims=True)
    unit_vectors = poses / (magnitudes+1e-10)
    output_array = np.concatenate((unit_vectors, magnitudes), axis=2)
    converted_data = np.concatenate((smplx_data[:, :6], output_array.reshape(poses.shape[0], -1), smplx_data[:, 171:187]), axis=1)
    return converted_data

In [65]:
convert_smplx_angle_rot(train_data[0][0]).shape

(300, 242)

In [11]:
all_train_data = np.concatenate([np.concatenate((T, D)) for T, D in train_data])

In [12]:
all_train_data.shape

(596400, 187)

In [14]:
train_mean = all_train_data.mean(axis=0)
train_std = all_train_data.std(axis=0)

In [15]:
def normalize_data(data, mean, std):
    return (data - mean) / std

In [17]:
normalized_train_data = [(normalize_data(T, train_mean, train_std), normalize_data(D, train_mean, train_std)) for T, D in train_data]

/var/folders/pj/3q7s4h6102n3v3kt6t8p1kzr0000gn/T/ipykernel_23897/1242856855.py:2: RuntimeWarning: invalid value encountered in divide
  return (data - mean) / std


In [20]:
all_test_data = np.concatenate([np.concatenate((T, D)) for T, D in test_data])

In [21]:
all_test_data.shape

(151800, 187)

In [22]:
test_mean = all_test_data.mean(axis=0)
test_std = all_test_data.std(axis=0)

In [23]:
normalized_test_data = [(normalize_data(T, train_mean, train_std), normalize_data(D, train_mean, train_std)) for T, D in test_data]

/var/folders/pj/3q7s4h6102n3v3kt6t8p1kzr0000gn/T/ipykernel_23897/1242856855.py:2: RuntimeWarning: invalid value encountered in divide
  return (data - mean) / std


In [46]:
normalized_train_transl_beta = [(np.concatenate([T[:, :3], T[:, 171:187]], axis=1),
                                 np.concatenate([D[:, :3], D[:, 171:187]], axis=1))
                                  for T, D in normalized_train_data]

In [49]:
normalized_train_global_orient_poses = [(T[:, 3:171], D[:, 3:171]) for T, D in normalized_train_data]

In [51]:
normalized_test_transl_beta = [(np.concatenate([T[:, :3], T[:, 171:187]], axis=1),
                                 np.concatenate([D[:, :3], D[:, 171:187]], axis=1))
                                  for T, D in normalized_test_data]

In [52]:
normalized_test_global_orient_poses = [(T[:, 3:171], D[:, 3:171]) for T, D in normalized_test_data]

In [54]:
def foot_contact_loss(smplx_data):
    '''smplx_data is of the shape (N, T, 187) Numpy.
    '''
    pos_3d_data = np.array([smplx_to_pos3d(data) for data in smplx_data])
    pos_3d_data = pos_3d_data.reshape(-1, 300, 55, 3)
    left_foot_z = pos_3d_data[:, :, 10, 3]
    right_foot_z = pos_3d_data[:, :, 13, 3]
    loss = (left_foot_z-0.0)**2 + (right_foot_z-0.0)**2
    return loss